In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Supress auto scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Load Datasets

### Top airbnb zipcodes throughout nyc
Get zip codes that need to be mapped to zillow home values

In [3]:
nyc_top_zipcodes_all_years = pd.read_csv("./zipcode_data/nyc_top_zipcodes_all_years.csv")
display(nyc_top_zipcodes_all_years.head())

,zipcode,borough,year,count
0,10469,BRONX,2022,122
1,10466,BRONX,2022,119
2,10463,BRONX,2018,105
3,10466,BRONX,2021,99
4,10456,BRONX,2022,98


### Zillow home value prices
Zillow home value data for each zipcode in nyc years 2002-2022

In [4]:
zillow_home_value = pd.read_csv('../data/clean/zhome_value.csv', low_memory=False)
zillow_home_value["date"] = pd.to_datetime(zillow_home_value["date"]).apply(lambda x: x.year)
zillow_home_value = zillow_home_value.rename(columns={"date": "year", "zip_code": "zipcode"})
display(zillow_home_value.head())

,zipcode,state,city,borough,year,home_value
0,11368,NY,New York,QUEENS,2002,249885.000
1,11385,NY,New York,QUEENS,2002,333404.000
2,11236,NY,New York,BROOKLYN,2002,255687.000
3,11208,NY,New York,BROOKLYN,2002,235676.000
4,10467,NY,New York,BRONX,2002,174696.000


In [5]:
zillow_home_value.groupby("borough")['home_value'].describe()

,count,mean,std,min,25%,50%,75%,max
borough,,,,,,,,
BRONX,5879.000,375671.926,120191.696,87134.000,298161.500,366814.000,446965.000,768859.000
BROOKLYN,9676.000,665321.092,293408.912,190659.000,450493.000,584672.000,845576.250,1571614.000
MANHATTAN,10357.000,1144362.172,610271.485,190835.000,731737.000,1001124.000,1378223.000,3633128.000
QUEENS,14151.000,524326.055,163166.506,183601.000,403074.500,494471.000,611980.500,1123137.000
STATEN_ISLAND,3000.000,443508.583,101820.704,196267.000,378883.500,432149.000,512108.500,780462.000


### Filter by significant years from top zipcodes
Find min and max year for top zipcodes that need to be mapped from "nyc_top_zipcodes_all_years"

In [6]:
min_year, max_year = nyc_top_zipcodes_all_years["year"].min(), nyc_top_zipcodes_all_years["year"].max()
print("MIN={}\nMAX={}".format(min_year, max_year))

MIN=2015
MAX=2022


Using **min,max** years from nyc_top_zipcodes, narrow down zillow dataset years to min, max years

In [7]:
zillow_home_value = zillow_home_value[zillow_home_value["year"].between(min_year, max_year, inclusive=True)]
display(zillow_home_value)

C:\Users\pumah\AppData\Local\Temp\ipykernel_21120\1115588723.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  zillow_home_value = zillow_home_value[zillow_home_value["year"].between(min_year, max_year, inclusive=True)]


,zipcode,state,city,borough,year,home_value
26486,11368,NY,New York,QUEENS,2015,417755.000
26487,11385,NY,New York,QUEENS,2015,537186.000
26488,11236,NY,New York,BROOKLYN,2015,401310.000
26489,11208,NY,New York,BROOKLYN,2015,396395.000
26490,10467,NY,New York,BRONX,2015,278072.000
...,...,...,...,...,...,...
43058,10464,NY,New York,BRONX,2022,609781.000
43059,10004,NY,New York,MANHATTAN,2022,1081807.000
43060,10006,NY,New York,MANHATTAN,2022,902566.000
43061,11243,NY,New York,BROOKLYN,2022,1426544.000


### Group by zipcode, year, borough and compute the average for home value

In [17]:
zillow_home_value = zillow_home_value.groupby(["zipcode", "year", "borough"])["home_value"].mean().to_frame().reset_index()
display(zillow_home_value)

,zipcode,year,borough,home_value
0,10001,2015,MANHATTAN,2280604.000
1,10001,2016,MANHATTAN,2369453.250
2,10001,2017,MANHATTAN,2271803.500
3,10001,2018,MANHATTAN,2351996.833
4,10001,2019,MANHATTAN,2380720.417
...,...,...,...,...
1407,11694,2018,QUEENS,769236.167
1408,11694,2019,QUEENS,819271.333
1409,11694,2020,QUEENS,829625.583
1410,11694,2021,QUEENS,897179.167


In [9]:
def merge_zipcode_home_value(zipcodes, zillow_home_value, boro):
    # Narrow down by borough
    merge_cols = ['zipcode','year']
    boro_zipcodes = zipcodes[zipcodes["borough"] == boro]
    # And remove "borough" column to prevent duplicates on merge
    boro_home_value = zillow_home_value[zillow_home_value["borough"]== boro].drop(["borough"], axis=1)
    
    merged_df = pd.merge(boro_zipcodes, boro_home_value,  how='left', left_on=merge_cols, right_on=merge_cols)
    return merged_df

# Non - intersecting
non_intersecting = zillow_home_value.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

In [51]:
bronx = non_intersecting[non_intersecting["borough"] == "BRONX"]
display(bronx)

,zipcode,year,borough,home_value,count
452,10451,2015,BRONX,252404.750,NaN
453,10451,2016,BRONX,226425.667,NaN
454,10451,2017,BRONX,251161.417,NaN
455,10451,2018,BRONX,263863.833,NaN
456,10451,2019,BRONX,313653.417,NaN
...,...,...,...,...,...
639,10475,2018,BRONX,445714.000,NaN
640,10475,2019,BRONX,489440.500,NaN
641,10475,2020,BRONX,519511.417,NaN
642,10475,2021,BRONX,537935.083,NaN


In [52]:
bronx.to_csv("bronx_remaining_zipcodes.csv", index=False)

In [53]:
bronx_csv = pd.read_csv("bronx_remaining_zipcodes.csv")

In [54]:
bronx_csv

,zipcode,year,borough,home_value,count
0,10451,2015,BRONX,252404.750,NaN
1,10451,2016,BRONX,226425.667,NaN
2,10451,2017,BRONX,251161.417,NaN
3,10451,2018,BRONX,263863.833,NaN
4,10451,2019,BRONX,313653.417,NaN
...,...,...,...,...,...
182,10475,2018,BRONX,445714.000,NaN
183,10475,2019,BRONX,489440.500,NaN
184,10475,2020,BRONX,519511.417,NaN
185,10475,2021,BRONX,537935.083,NaN


# Map home value to top airbnb zipcodes for each borough in NYC

Helper function to narrow down each data frame to borough and map home prices for that year

In [9]:
def merge_zipcode_home_value(zipcodes, zillow_home_value, boro):
    # Narrow down by borough
    merge_cols = ['zipcode','year']   
    boro_zipcodes = zipcodes[zipcodes["borough"] == boro]
    # And remove "borough" column to prevent duplicates on merge
    boro_home_value = zillow_home_value[zillow_home_value["borough"]== boro].drop(["borough"], axis=1)
    
    merged_df = pd.merge(boro_zipcodes, boro_home_value,  how='left', left_on=merge_cols, right_on=merge_cols)
    return merged_df

Display mapped home values for each top zipcode in each borough, also create cumulative df "nyc_top_zipcode_home_val"

In [10]:
boros = nyc_top_zipcodes_all_years["borough"].unique().tolist()
all_boro_home_value = pd.DataFrame()

for borough in boros:
    boro_data = merge_zipcode_home_value(nyc_top_zipcodes_all_years, zillow_home_value, borough).sort_values(by=["home_value"], ascending=False)
    print("Borough={}".format(borough))
    display(boro_data)
    all_boro_home_value = pd.concat([all_boro_home_value, boro_data])

Borough=BRONX


,zipcode,borough,year,count,home_value
0,10469,BRONX,2022,122,592184.800
1,10466,BRONX,2022,119,560980.300
3,10466,BRONX,2021,99,528377.083
4,10456,BRONX,2022,98,454902.900
2,10463,BRONX,2018,105,357383.417


Borough=BROOKLYN


,zipcode,borough,year,count,home_value
2,11211,BROOKLYN,2022,2730,1150638.400
3,11211,BROOKLYN,2020,2719,1036530.250
0,11211,BROOKLYN,2019,6231,1023273.750
1,11211,BROOKLYN,2018,3227,997202.917
4,11211,BROOKLYN,2017,2645,967886.000


Borough=MANHATTAN


,zipcode,borough,year,count,home_value
3,10013,MANHATTAN,2019,1779,3310300.500
4,10024,MANHATTAN,2019,1721,1726462.083
1,10018,MANHATTAN,2018,2485,1463682.250
2,10003,MANHATTAN,2017,2184,1402998.500
0,10009,MANHATTAN,2017,2593,1091941.333


Borough=QUEENS


,zipcode,borough,year,count,home_value
4,11103,QUEENS,2018,443,881343.583
1,11103,QUEENS,2017,584,822617.000
0,11106,QUEENS,2018,648,618868.333
3,11104,QUEENS,2019,465,578418.083
2,11106,QUEENS,2017,531,576068.750


Borough=STATEN ISLAND


,zipcode,borough,year,count,home_value
0,10301,STATEN ISLAND,2019,212,NaN
1,10301,STATEN ISLAND,2022,143,NaN
2,10301,STATEN ISLAND,2021,126,NaN
3,10301,STATEN ISLAND,2015,124,NaN
4,10301,STATEN ISLAND,2016,118,NaN


### NYC data for all top airbnb zipcodes and their home values 2002 - 2022

In [11]:
display(all_boro_home_value)

,zipcode,borough,year,count,home_value
0,10469,BRONX,2022,122,592184.800
1,10466,BRONX,2022,119,560980.300
3,10466,BRONX,2021,99,528377.083
4,10456,BRONX,2022,98,454902.900
2,10463,BRONX,2018,105,357383.417
2,11211,BROOKLYN,2022,2730,1150638.400
3,11211,BROOKLYN,2020,2719,1036530.250
0,11211,BROOKLYN,2019,6231,1023273.750
1,11211,BROOKLYN,2018,3227,997202.917
4,11211,BROOKLYN,2017,2645,967886.000


### Test for normality

In [12]:
# Generate Shapiro wilk test for 
def generate_shapiro_test(all_boro_home_value, boro):
    data = all_boro_home_value[all_boro_home_value["borough"] == boro]
    shapiro_wilk_data = stats.shapiro(data['home_value'])
    print(boro)
    print("W={}, P={}".format(shapiro_wilk_data[0], shapiro_wilk_data[1]))

In [13]:
generate_shapiro_test(all_boro_home_value, "BRONX")
generate_shapiro_test(all_boro_home_value, "BROOKLYN")
generate_shapiro_test(all_boro_home_value, "QUEENS")
generate_shapiro_test(all_boro_home_value, "MANHATTAN")

BRONX
W=0.9325533509254456, P=0.6138831973075867
BROOKLYN
W=0.8819568157196045, P=0.3182726502418518
QUEENS
W=0.8109543323516846, P=0.09922949224710464
MANHATTAN
W=0.7880370020866394, P=0.06450841575860977


### Get Remaining Zip codes (with top 5 excluded) for each borough

In [14]:
# Non - intersecting zipcodes: Allzipcodes - top 5 zipcodes
remaining_zipcodes = zillow_home_value.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

C:\Users\pumah\AppData\Local\Temp\ipykernel_21120\3189024214.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  remaining_zipcodes = zillow_home_value.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)


In [15]:
def get_remaining_zipcodes_zipcodes(remaining_zipcodes, boro):
    return remaining_zipcodes[remaining_zipcodes["borough"] == boro]

In [16]:
# Non intersecting zipcodes
def create_remaining_zipcode_csvs(remaining_zipcodes, display=False):
    boros = ["BRONX", "BROOKLYN", "QUEENS", "MANHATTAN", "STATEN_ISLAND"]
    for boro in boros:
        data = get_remaining_zipcodes_zipcodes(remaining_zipcodes, boro)
        create_csv(data, "remaining_zipcodes", "remaining_zipcode_home_value", boro, display)

In [17]:
# create_remaining_zipcode_csvs(remaining_zipcodes, True)

In [18]:
remaining_zipcode_count = pd.read_csv("./zipcode_data/nyc_remaining_zipcode_count.csv")

In [19]:
print("Remaining zipcodes count:", len(remaining_zipcode_count["zipcode"].unique().tolist()))

Remaining zipcodes count: 186


In [20]:
print("Zillow zipcode count:", len(zillow_home_value["zipcode"].unique().tolist()))

Zillow zipcode count: 177


In [21]:
merge_cols = ["year", "zipcode"]
remaining_mapped = pd.merge(remaining_zipcode_count, zillow_home_value,  how='left', left_on=merge_cols, right_on=merge_cols)

In [22]:
remaining_mapped

,zipcode,borough_x,year,count,borough_y,home_value
0,10001,MANHATTAN,2015,350,MANHATTAN,2280604.000
1,10001,MANHATTAN,2016,666,MANHATTAN,2369453.250
2,10001,MANHATTAN,2017,418,MANHATTAN,2271803.500
3,10001,MANHATTAN,2018,491,MANHATTAN,2351996.833
4,10001,MANHATTAN,2019,275,MANHATTAN,2380720.417
...,...,...,...,...,...,...
1399,11694,QUEENS,2020,11,QUEENS,829625.583
1400,11694,QUEENS,2021,43,QUEENS,897179.167
1401,11694,QUEENS,2022,64,QUEENS,955955.100
1402,11697,QUEENS,2016,1,NaN,NaN


In [25]:
# Rockerfeller center 10020, check what type of areas we're missing.
# Look for significant counts andor/ places
remaining_mapped[remaining_mapped["home_value"].isnull()]   

,zipcode,borough_x,year,count,borough_y,home_value
108,10020,MANHATTAN,2015,460,NaN,NaN
109,10020,MANHATTAN,2016,458,NaN,NaN
110,10020,MANHATTAN,2017,219,NaN,NaN
111,10020,MANHATTAN,2018,253,NaN,NaN
112,10020,MANHATTAN,2019,533,NaN,NaN
...,...,...,...,...,...,...
1327,11430,QUEENS,2022,2,NaN,NaN
1368,11580,QUEENS,2021,7,NaN,NaN
1369,11580,QUEENS,2022,7,NaN,NaN
1402,11697,QUEENS,2016,1,NaN,NaN
